In [1]:
year = 2024

month_num_to_abbr = {
    1: 'jan',
    2: 'feb',
    3: 'mar',
    4: 'apr',
    5: 'may',
    6: 'jun',
    7: 'jul',
    8: 'aug',
    9: 'sep',
    10: 'oct',
    11: 'nov',
    12: 'dec'
}

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from random import randint
from time import time,sleep
from datetime import datetime, timedelta
import schedule
from pathlib import Path  

def scrape_forex_data(month_num):

    # This section opens the browser using selenium webdriver and navigates to the this week tab
    URL = f"https://www.forexfactory.com/calendar?month={month_num_to_abbr[month_num]}.{year}"

    #open the browser
    browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    #load page
    browser.get(URL)

    # Wait for the table element to load
    table = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "calendar__table  "))
    )
    
    sleep(1)
    
    # Parse the HTML with Beautiful Soup
    soup = BeautifulSoup(browser.page_source, "html.parser") 
    
    
    # Extract the data from the table
    previous_date = ""
    d = {'Date': [], 'Time': [], 'Currency': [], 'Impact': [],'Description': [], 'Actual': [], 'Forecast': [], 'Previous': []}
    for row in soup.find_all("tr"):
        if "calendar__row" in row.get("class", []):
            date_cell = row.find("td", {"class": "calendar__date"})
            if date_cell is not None and date_cell.text.strip() != "":
                date = date_cell.text.strip()
                d['Date'].append(date)
                previous_date = date
            else:
                d['Date'].append(previous_date)
            time_cell = row.find("td", {"class": "calendar__time"})
            if time_cell is not None:
                d['Time'].append( time_cell.text.strip())
            else:
                d['Time'].append("")
            currency_cell = row.find("td", {"class": "calendar__currency"})
            if currency_cell is not None:
                d['Currency'].append(currency_cell.text.strip())
            else:
                d['Currency'].append( "")
            actual_cell = row.find("td", {"class": "calendar__actual"})
            if actual_cell is not None:
                d['Actual'].append(actual_cell.text.strip())
            else:
                d['Actual'].append("")
            forecast_cell = row.find("td", {"class": "calendar__forecast"})
            if forecast_cell is not None:
                d['Forecast'].append(forecast_cell.text.strip())
            else:
                d['Forecast'].append("")
            previous_cell = row.find("td", {"class": "calendar__previous"})
            if previous_cell is not None:
                d['Previous'].append(previous_cell.text.strip())
            else:
                d['Previous'].append("")
            event_cell = row.find("td", {"class": "calendar__event"})
            if event_cell is not None:
                d['Description'].append(event_cell.text.strip())
            else:
                d['Description'].append( "")
            impact_cell = row.find("td", {"class": "calendar__impact"})
            if impact_cell is not None:
                impact = impact_cell.find('span', {'title': True})
                if impact is not None:
                    d['Impact'].append(impact['title'])
                else:
                    d['Impact'].append("")
            else:
                d['Impact'].append("")
                


    forex_factory_df = pd.DataFrame.from_dict(d) 
    forex_factory_df['Time'].replace('', pd.NA, inplace=True)
    forex_factory_df['Time'].fillna(method='ffill', inplace=True)

    # Remove rows that only have date information
    forex_factory_df = forex_factory_df[forex_factory_df['Currency'] != '']
    
#     display(forex_factory_df)

    path = fr"C:\Users\user\Stocks\data\dataset-forexfactory-main\{year}\{month_num}\forex_factory.csv"
    filepath = Path(path)
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    forex_factory_df.to_csv(filepath, header=True, index=False)  

#     filename = f'forex_factory.csv'
#     #insert path of folder you want the csv to be in between r''
#     forex_factory_df.to_csv(r'1' + '\\' + filename, header=True, index=False) 

    # Close the web driver
    browser.quit()

In [3]:
for i in range(1, 13):
    scrape_forex_data(i)

,Date,Time,Currency,Impact,Description,Actual,Forecast,Previous
1,Mon Jan 1,All Day,NZD,Non-Economic,Bank Holiday,,,
2,Mon Jan 1,All Day,AUD,Non-Economic,Bank Holiday,,,
3,Mon Jan 1,All Day,JPY,Non-Economic,Bank Holiday,,,
4,Mon Jan 1,All Day,CNY,Non-Economic,Bank Holiday,,,
5,Mon Jan 1,All Day,CHF,Non-Economic,Bank Holiday,,,
6,Mon Jan 1,All Day,EUR,Non-Economic,French Bank Holiday,,,
7,Mon Jan 1,All Day,EUR,Non-Economic,German Bank Holiday,,,
8,Mon Jan 1,All Day,EUR,Non-Economic,Italian Bank Holiday,,,
9,Mon Jan 1,All Day,GBP,Non-Economic,Bank Holiday,,,
10,Mon Jan 1,All Day,CAD,Non-Economic,Bank Holiday,,,


,Date,Time,Currency,Impact,Description,Actual,Forecast,Previous
1,Thu Feb 1,3:00am,USD,High Impact Expected,Federal Funds Rate,5.50%,5.50%,5.50%
2,Thu Feb 1,3:00am,USD,High Impact Expected,FOMC Statement,,,
3,Thu Feb 1,3:30am,USD,High Impact Expected,FOMC Press Conference,,,
4,Thu Feb 1,8:30am,AUD,Low Impact Expected,Building Approvals m/m,-9.5%,0.5%,0.3%
5,Thu Feb 1,8:30am,AUD,Low Impact Expected,Import Prices q/q,1.1%,0.6%,0.8%
6,Thu Feb 1,8:30am,AUD,Low Impact Expected,NAB Quarterly Business Confidence,-6,,-1
7,Thu Feb 1,8:30am,JPY,Low Impact Expected,Final Manufacturing PMI,48.0,48.0,48.0
8,Thu Feb 1,9:45am,CNY,Medium Impact Expected,Caixin Manufacturing PMI,50.8,50.8,50.8
9,Thu Feb 1,11:35am,JPY,Low Impact Expected,10-y Bond Auction,0.74|3.6,,0.60|2.9
10,Thu Feb 1,1:30pm,AUD,Low Impact Expected,Commodity Prices y/y,-10.4%,,-11.2%


,Date,Time,Currency,Impact,Description,Actual,Forecast,Previous
1,Fri Mar 1,Day 2,All,Medium Impact Expected,G20 Meetings,,,
2,Fri Mar 1,2:15am,USD,Medium Impact Expected,FOMC Member Mester Speaks,,,
3,Fri Mar 1,5:45am,NZD,Low Impact Expected,Building Consents m/m,-8.8%,,3.6%
4,Fri Mar 1,7:30am,JPY,Low Impact Expected,Unemployment Rate,2.4%,2.4%,2.4%
5,Fri Mar 1,8:05am,NZD,Medium Impact Expected,RBNZ Gov Orr Speaks,,,
6,Fri Mar 1,8:30am,JPY,Low Impact Expected,Final Manufacturing PMI,47.2,47.2,47.2
7,Fri Mar 1,9:10am,USD,Low Impact Expected,FOMC Member Williams Speaks,,,
8,Fri Mar 1,9:30am,CNY,High Impact Expected,Manufacturing PMI,49.1,49.1,49.2
9,Fri Mar 1,9:30am,CNY,Medium Impact Expected,Non-Manufacturing PMI,51.4,50.9,50.7
10,Fri Mar 1,9:45am,CNY,Medium Impact Expected,Caixin Manufacturing PMI,50.9,50.7,50.8


,Date,Time,Currency,Impact,Description,Actual,Forecast,Previous
1,Mon Apr 1,All Day,NZD,Non-Economic,Bank Holiday,,,
2,Mon Apr 1,All Day,AUD,Non-Economic,Bank Holiday,,,
3,Mon Apr 1,7:50am,JPY,Low Impact Expected,Tankan Manufacturing Index,11,10,12
4,Mon Apr 1,7:50am,JPY,Low Impact Expected,Tankan Non-Manufacturing Index,34,31,30
5,Mon Apr 1,8:30am,JPY,Low Impact Expected,Final Manufacturing PMI,48.2,48.2,48.2
6,Mon Apr 1,9:45am,CNY,Low Impact Expected,Caixin Manufacturing PMI,51.1,51.0,50.9
7,Mon Apr 1,All Day,CHF,Non-Economic,Bank Holiday,,,
8,Mon Apr 1,All Day,EUR,Non-Economic,French Bank Holiday,,,
9,Mon Apr 1,All Day,EUR,Non-Economic,German Bank Holiday,,,
10,Mon Apr 1,All Day,EUR,Non-Economic,Italian Bank Holiday,,,


,Date,Time,Currency,Impact,Description,Actual,Forecast,Previous
1,Wed May 1,5:00am,NZD,Low Impact Expected,RBNZ Financial Stability Report,,,
2,Wed May 1,6:45am,NZD,High Impact Expected,Employment Change q/q,-0.2%,0.3%,0.4%
3,Wed May 1,6:45am,NZD,High Impact Expected,Unemployment Rate,4.3%,4.3%,4.0%
4,Wed May 1,6:45am,NZD,Low Impact Expected,Labor Cost Index q/q,0.8%,0.8%,1.0%
5,Wed May 1,All Day,CNY,Non-Economic,Bank Holiday,,,
6,Wed May 1,8:30am,JPY,Low Impact Expected,Final Manufacturing PMI,49.6,49.9,49.9
7,Wed May 1,9:00am,NZD,Medium Impact Expected,RBNZ Gov Orr Speaks,,,
8,Wed May 1,All Day,CHF,Non-Economic,Bank Holiday,,,
9,Wed May 1,2:00pm,GBP,Low Impact Expected,Nationwide HPI m/m,-0.4%,0.1%,-0.2%
10,Wed May 1,All Day,EUR,Non-Economic,French Bank Holiday,,,


...

,Date,Time,Currency,Impact,Description,Actual,Forecast,Previous
1,Thu Aug 1,2:00am,USD,High Impact Expected,Federal Funds Rate,5.50%,5.50%,5.50%
2,Thu Aug 1,2:00am,USD,High Impact Expected,FOMC Statement,,,
3,Thu Aug 1,2:30am,USD,High Impact Expected,FOMC Press Conference,,,
4,Thu Aug 1,8:30am,JPY,Low Impact Expected,Final Manufacturing PMI,49.1,49.2,49.2
5,Thu Aug 1,9:30am,AUD,Low Impact Expected,Goods Trade Balance,5.59B,5.08B,5.05B
6,Thu Aug 1,9:30am,AUD,Low Impact Expected,Import Prices q/q,1.0%,-0.9%,-1.8%
7,Thu Aug 1,9:45am,CNY,Low Impact Expected,Caixin Manufacturing PMI,49.8,51.4,51.8
8,Thu Aug 1,All Day,CHF,Non-Economic,Bank Holiday,,,
9,Thu Aug 1,2:00pm,GBP,Low Impact Expected,Nationwide HPI m/m,0.3%,0.1%,0.2%
10,Thu Aug 1,2:30pm,AUD,Low Impact Expected,Commodity Prices y/y,-3.0%,,-3.5%


,Date,Time,Currency,Impact,Description,Actual,Forecast,Previous
3,Mon Sep 2,7:50am,JPY,Low Impact Expected,Capital Spending q/y,7.4%,9.6%,6.8%
4,Mon Sep 2,8:30am,JPY,Low Impact Expected,Final Manufacturing PMI,49.8,49.5,49.5
5,Mon Sep 2,9:00am,AUD,Low Impact Expected,MI Inflation Gauge m/m,-0.1%,,0.4%
6,Mon Sep 2,9:30am,AUD,Low Impact Expected,ANZ Job Advertisements m/m,-2.1%,,-2.7%
7,Mon Sep 2,9:30am,AUD,Low Impact Expected,Building Approvals m/m,10.4%,2.4%,-6.4%
8,Mon Sep 2,9:30am,AUD,Low Impact Expected,Company Operating Profits q/q,-5.3%,-0.6%,-2.5%
9,Mon Sep 2,9:45am,CNY,Low Impact Expected,Caixin Manufacturing PMI,50.4,50.0,49.8
10,Mon Sep 2,2:30pm,CHF,Low Impact Expected,Retail Sales y/y,2.7%,-0.2%,-2.6%
11,Mon Sep 2,2:32pm,AUD,Low Impact Expected,Commodity Prices y/y,-5.2%,,-4.2%
12,Mon Sep 2,3:15pm,EUR,Low Impact Expected,Spanish Manufacturing PMI,50.5,51.4,51.0


,Date,Time,Currency,Impact,Description,Actual,Forecast,Previous
1,Tue Oct 1,5:45am,NZD,Low Impact Expected,Building Consents m/m,,,
2,Tue Oct 1,7:50am,JPY,Low Impact Expected,BOJ Summary of Opinions,,,
3,Tue Oct 1,All Day,CNY,Non-Economic,Bank Holiday,,,
4,Tue Oct 1,8:30am,JPY,Low Impact Expected,Final Manufacturing PMI,,,
5,Tue Oct 1,9:30am,AUD,Medium Impact Expected,Retail Sales m/m,,,
6,Tue Oct 1,9:30am,AUD,Low Impact Expected,Building Approvals m/m,,,
7,Tue Oct 1,9:45am,CNY,Low Impact Expected,Caixin Manufacturing PMI,,,
8,Tue Oct 1,2:30pm,AUD,Low Impact Expected,Commodity Prices y/y,,,
9,Tue Oct 1,2:45pm,EUR,Low Impact Expected,French Gov Budget Balance,,,
10,Tue Oct 1,3:15pm,EUR,Low Impact Expected,Spanish Manufacturing PMI,,,


,Date,Time,Currency,Impact,Description,Actual,Forecast,Previous
1,Fri Nov 1,4:00am,NZD,Low Impact Expected,RBNZ Financial Stability Report,,,
2,Fri Nov 1,5:45am,NZD,Low Impact Expected,Building Consents m/m,,,
3,Fri Nov 1,8:30am,AUD,Low Impact Expected,Building Approvals m/m,,,
4,Fri Nov 1,8:30am,AUD,Low Impact Expected,PPI q/q,,,
5,Fri Nov 1,8:30am,JPY,Low Impact Expected,Final Manufacturing PMI,,,
6,Fri Nov 1,9:45am,CNY,Low Impact Expected,Caixin Manufacturing PMI,,,
7,Fri Nov 1,1:30pm,AUD,Low Impact Expected,Commodity Prices y/y,,,
8,Fri Nov 1,All Day,EUR,Non-Economic,French Bank Holiday,,,
9,Fri Nov 1,All Day,EUR,Non-Economic,Italian Bank Holiday,,,
10,Fri Nov 1,2:45pm,EUR,Low Impact Expected,French Gov Budget Balance,,,


,Date,Time,Currency,Impact,Description,Actual,Forecast,Previous
1,Sun Dec 1,All Day,All,Medium Impact Expected,OPEC-JMMC Meetings,,,
2,Sun Dec 1,All Day,All,Medium Impact Expected,OPEC Meetings,,,
4,Mon Dec 2,5:45am,NZD,Low Impact Expected,Building Consents m/m,,,
5,Mon Dec 2,5:45am,NZD,Low Impact Expected,Overseas Trade Index q/q,,,
6,Mon Dec 2,7:30am,JPY,Low Impact Expected,Unemployment Rate,,,
7,Mon Dec 2,7:50am,JPY,Low Impact Expected,Capital Spending q/y,,,
8,Mon Dec 2,8:30am,AUD,Medium Impact Expected,Retail Sales m/m,,,
9,Mon Dec 2,8:30am,AUD,Low Impact Expected,Building Approvals m/m,,,
10,Mon Dec 2,8:30am,AUD,Low Impact Expected,Company Operating Profits q/q,,,
11,Mon Dec 2,8:30am,JPY,Low Impact Expected,Final Manufacturing PMI,,,
